In [1]:
import os, sys
root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

import Auto3D
from Auto3D.auto3D import options, main

In [2]:
#Always ensure that you have the latest version
print(Auto3D.__version__)

1.15


# Generate low-energy 3D structures with Auto3D

In [3]:
if __name__ == "__main__":
    path = os.path.join(root, "example/files/smiles.smi")
    args = options(path, k=1, use_gpu=False)   #args specify the parameters for Auto3D 
    out = main(args)            #main acceps the parameters and run Auto3D
    print(out)

Checking input file...
	There are 4 SMILES in the input file /home/jack/Auto3D_pkg/example/files/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI2xt and AIMNET.
The available memory is 125 GB.
The task will be divided into 1 jobs.
Job1, number of inputs: 4


Isomer generation for job1
Enumerating cis/tran isomers for unspecified double bonds...
Enumerating R/S isomers for unspecified atomic centers...
Removing enantiomers...
Enumerating conformers/rotamers, removing duplicates...


Optimizing on job1
Preparing for parallel optimizing... (Max optimization steps: 5000)
Total 3D conformers: 13


 10%|▉         | 497/5000 [00:07<00:44, 100.29it/s]

Total 3D structures: 13  Converged: 9   Dropped(Oscillating): 0    Active: 4


 20%|█▉        | 996/5000 [00:11<00:32, 121.72it/s]

Total 3D structures: 13  Converged: 12   Dropped(Oscillating): 0    Active: 1


 24%|██▍       | 1208/5000 [00:13<00:41, 91.98it/s] 


Optimization finished at step 1209:   Total 3D structures: 13  Converged: 13   Dropped(Oscillating): 0    Active: 0
Beggin to slelect structures that satisfy the requirements...
Energy unit: Hartree if implicit.
Program running time: 0 minutes
Output path: /home/jack/Auto3D_pkg/example/files/20221216-163935-396901_smiles/smiles_out.sdf
/home/jack/Auto3D_pkg/example/files/20221216-163935-396901_smiles/smiles_out.sdf
